In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install medmnist kaggle opencv-python

In [3]:
!pip install torch torchvision medmnist scikit-learn determined torch torchvision

In [4]:
# %% [code]
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import albumentations as A
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from medmnist import OCTMNIST, BreastMNIST, PneumoniaMNIST, RetinaMNIST, INFO

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

# %% [markdown]
# ## 1. Data Loading and Preprocessing
# 
# Define functions:
# - `preprocess_general`: For OCTMNIST, PneumoniaMNIST, and BreastMNIST.
# - `preprocess_retina`: For RetinaMNIST (converts to grayscale, applies median filter and CLAHE).

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
# %% [code]
def preprocess_general(img):
    if img.ndim == 3 and img.shape[-1] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    elif img.ndim == 3 and img.shape[-1] == 1:
        img = img.squeeze()
    return img.astype(np.float32) / 255.0

def preprocess_retina(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    denoised = cv2.medianBlur(gray, 3)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    proc = clahe.apply(denoised)
    return proc.astype(np.float32) / 255.0


In [6]:
# %% [code]
def load_and_preprocess(dataset_cls, offset, is_retina=False):
    X_train, y_train, X_val, y_val, X_test, y_test = [], [], [], [], [], []
    for split in ['train', 'val', 'test']:
        dataset = dataset_cls(split=split, download=True)
        for img, label in tqdm(zip(dataset.imgs, dataset.labels.squeeze()), total=len(dataset.imgs), desc=f"Processing {dataset_cls.__name__} {split}"):
            label += offset  # ensure unique label ranges
            proc_img = preprocess_retina(img) if is_retina else preprocess_general(img)
            if split == 'train':
                X_train.append(proc_img)
                y_train.append(label)
            elif split == 'val':
                X_val.append(proc_img)
                y_val.append(label)
            else:
                X_test.append(proc_img)
                y_test.append(label)
    return X_train, y_train, X_val, y_val, X_test, y_test


In [7]:
# %% [markdown]
# **Assign Label Offsets:**  
# - OCTMNIST: 0–3  
# - PneumoniaMNIST: next 2 classes  
# - RetinaMNIST: next 5 classes  
# - BreastMNIST: last 2 classes  
# This gives a total of 4+2+5+2 = 13 classes.
# %% [code]
offsets = {
    'octmnist': 0,
    'pneumoniamnist': len(INFO['octmnist']['label']),              # typically 4
    'retinamnist': len(INFO['octmnist']['label']) + len(INFO['pneumoniamnist']['label']),  # 4+2=6
    'breastmnist': len(INFO['octmnist']['label']) + len(INFO['pneumoniamnist']['label']) + len(INFO['retinamnist']['label'])  # 6+5=11
}

# Load datasets
X_train, y_train, X_val, y_val, X_test, y_test = [], [], [], [], [], []
datasets = [
    ('octmnist', OCTMNIST, False),
    ('pneumoniamnist', PneumoniaMNIST, False),
    ('retinamnist', RetinaMNIST, True),
    ('breastmnist', BreastMNIST, False)
]
for name, cls, is_retina in datasets:
    Xt, yt, Xv, yv, Xte, yte = load_and_preprocess(cls, offsets[name], is_retina)
    X_train += Xt; y_train += yt
    X_val   += Xv; y_val   += yv
    X_test  += Xte; y_test  += yte

# Convert lists to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val   = np.array(X_val)
y_val   = np.array(y_val)
X_test  = np.array(X_test)
y_test  = np.array(y_test)

print("Train images:", X_train.shape, "Train labels:", y_train.shape)
print("Validation images:", X_val.shape, "Validation labels:", y_val.shape)
print("Test images:", X_test.shape, "Test labels:", y_test.shape)


100%|██████████| 54.9M/54.9M [00:04<00:00, 11.1MB/s]
Processing OCTMNIST train: 100%|██████████| 97477/97477 [00:00<00:00, 118390.80it/s]


Using downloaded and verified file: /root/.medmnist/octmnist.npz


Processing OCTMNIST val: 100%|██████████| 10832/10832 [00:00<00:00, 118332.20it/s]


Using downloaded and verified file: /root/.medmnist/octmnist.npz


Processing OCTMNIST test: 100%|██████████| 1000/1000 [00:00<00:00, 115551.93it/s]


...

Using downloaded and verified file: /root/.medmnist/breastmnist.npz


Processing BreastMNIST val: 100%|██████████| 78/78 [00:00<00:00, 61704.21it/s]


Using downloaded and verified file: /root/.medmnist/breastmnist.npz


Processing BreastMNIST test: 100%|██████████| 156/156 [00:00<00:00, 86469.07it/s]


Train images: (103811, 28, 28) Train labels: (103811,)
Validation images: (11554, 28, 28) Validation labels: (11554,)
Test images: (2180, 28, 28) Test labels: (2180,)


In [8]:
# %% [markdown]
# ## 2. Data Augmentation
# 
# Augment classes in the training set with fewer than 2000 images (expand to 5× the original number).
# %% [code]
# Calculate class counts
unique, counts = np.unique(y_train, return_counts=True)
class_counts = dict(zip(unique, counts))
print("Initial class distribution:", class_counts)

# Identify classes to augment (less than 2000 images)
augment_classes = [cls for cls, cnt in class_counts.items() if cnt < 2000]
print("Classes to augment:", augment_classes)

# Define augmentation pipeline using Albumentations
# Define augmentation pipeline using Albumentations (without RandAugment)
augment_pipeline = A.Compose([
    A.RandomResizedCrop(28, 28, scale=(0.8, 1.0), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
    A.OneOf([
        A.Equalize(p=0.5),
        A.Solarize(p=0.5),
        A.Posterize(num_bits=4, p=0.5),
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.5),
    ], p=0.5)
])


aug_imgs, aug_labels = [], []
for cls in augment_classes:
    cls_idxs = np.where(y_train == cls)[0]
    cls_imgs = X_train[cls_idxs]
    target_count = 5 * len(cls_imgs)
    needed = target_count - len(cls_imgs)
    print(f"Augmenting class {cls}: need {needed} new samples")
    for _ in range(needed):
        img = cls_imgs[np.random.randint(len(cls_imgs))]
        # Albumentations expects uint8 images; convert from [0,1] float32
        aug_img = augment_pipeline(image=(img * 255).astype(np.uint8))['image']
        aug_img = aug_img.astype(np.float32) / 255.0
        aug_imgs.append(aug_img)
        aug_labels.append(cls)

if aug_imgs:
    X_train = np.concatenate([X_train, np.stack(aug_imgs)])
    y_train = np.concatenate([y_train, np.array(aug_labels)])
    print("After augmentation, training set shape:", X_train.shape)


Initial class distribution: {0: 33484, 1: 10213, 2: 7754, 3: 46026, 4: 1214, 5: 3494, 6: 486, 7: 128, 8: 206, 9: 194, 10: 66, 11: 147, 12: 399}
Classes to augment: [4, 6, 7, 8, 9, 10, 11, 12]
Augmenting class 4: need 4856 new samples
Augmenting class 6: need 1944 new samples
Augmenting class 7: need 512 new samples
Augmenting class 8: need 824 new samples
Augmenting class 9: need 776 new samples
Augmenting class 10: need 264 new samples
Augmenting class 11: need 588 new samples
Augmenting class 12: need 1596 new samples
After augmentation, training set shape: (115171, 28, 28)


In [9]:
# %% [markdown]
# ## 3. Create PyTorch Dataset and DataLoader
# 
# The dataset will reshape images to (1, 28, 28).
# %% [code]
class MedmnistDataset(Dataset):
    def __init__(self, images, labels):
        # images shape: (N, 28, 28); add channel dimension
        self.images = images.reshape(-1, 1, 28, 28).astype(np.float32)
        self.labels = labels.astype(np.int64)
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        return torch.tensor(self.images[idx]), torch.tensor(self.labels[idx])
    
batch_size = 53
train_dataset = MedmnistDataset(X_train, y_train)
val_dataset   = MedmnistDataset(X_val, y_val)
test_dataset  = MedmnistDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [10]:
# %% [markdown]
# ## 4. Define Custom ResNet-50-based Model
# 
# The model uses:
# - Base: ResNet-50  
# - Customizations:
#   - Adjusted first convolution layer to accept 1-channel input.
#   - Removed the max pooling layer.
#   - Final fully connected layer outputs 13 classes.
# %% [code]
from torchvision import models

class CustomResNet50(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet50, self).__init__()
        # Load ResNet-50 (without pre-trained weights)
        self.resnet = models.resnet50(pretrained=False)
        # Modify conv1 to accept 1-channel input
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Remove max pooling layer by replacing with identity
        self.resnet.maxpool = nn.Identity()
        # Modify final fully connected layer to output num_classes
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
    def forward(self, x):
        return self.resnet(x)

num_classes = 13
model = CustomResNet50(num_classes)
model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [11]:
# %% [markdown]
# ## 5. Training Setup
# 
# **Hyperparameters:**
# - Batch Size: 53  
# - Learning Rate: 0.01332344940133225  
# - Weight Decay: 0.00021921795989143406  
# - Optimizer: SGD with momentum 0.9  
# - Scheduler: ReduceLROnPlateau
# %% [code]
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01332344940133225, momentum=0.9, weight_decay=0.00021921795989143406)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [12]:
# %% [code]
import time

def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        # The model expects 1-channel input (28x28) so no channel replication is needed.
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    return running_loss / len(loader.dataset)

def evaluate_model(model, loader, device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_true, y_pred)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 10
best_val_acc = 0.0

for epoch in range(num_epochs):
    start = time.time()
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    val_acc = evaluate_model(model, val_loader, device)
    scheduler.step(val_acc)
    end = time.time()
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f} | Time: {end-start:.2f}s")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_custom_resnet50.pth")
        print("Saved best model.")


Epoch 1/10 | Loss: 0.8744 | Val Acc: 0.4962 | Time: 148.03s
Saved best model.
Epoch 2/10 | Loss: 0.5423 | Val Acc: 0.7989 | Time: 147.11s
Saved best model.
Epoch 3/10 | Loss: 0.4301 | Val Acc: 0.8528 | Time: 146.99s
Saved best model.
Epoch 4/10 | Loss: 0.4174 | Val Acc: 0.8694 | Time: 147.06s
Saved best model.
Epoch 5/10 | Loss: 0.3383 | Val Acc: 0.9023 | Time: 147.02s
Saved best model.
Epoch 6/10 | Loss: 0.3102 | Val Acc: 0.8988 | Time: 147.02s
Epoch 7/10 | Loss: 0.3120 | Val Acc: 0.9088 | Time: 147.02s
Saved best model.
Epoch 8/10 | Loss: 0.2754 | Val Acc: 0.9023 | Time: 146.97s
Epoch 9/10 | Loss: 0.2661 | Val Acc: 0.6613 | Time: 146.92s
Epoch 10/10 | Loss: 0.3200 | Val Acc: 0.8455 | Time: 146.84s


In [15]:
# %% [markdown]
# ## 7. Final Model Saving and Download Instructions
# 
# The best model is saved as `best_custom_resnet50.pth`.  
# In Kaggle's Files pane, download this model along with the evaluation script (see next cell).
# %% [markdown]
# ## 8. Create Evaluation Script for Local Use
# 
# This script (saved as `evaluate_model.py`) loads the saved model and evaluates on a provided CSV file.
# %% [code]
eval_script = r"""
import torch
import torch.nn as nn
from torchvision import models
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score

class MedmnistDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)
        self.X = df.drop('label', axis=1).values.reshape(-1, 1, 28, 28).astype(np.float32)
        self.y = df['label'].values.astype(np.int64)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])
    
class CustomResNet50(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet50, self).__init__()
        self.resnet = models.resnet50(pretrained=False)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.maxpool = nn.Identity()
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
    def forward(self, x):
        return self.resnet(x)

if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--model-path", type=str, default="best_custom_resnet50.pth", help="Path to the trained model")
    parser.add_argument("--csv-path", type=str, required=True, help="Path to the test CSV file")
    parser.add_argument("--num-classes", type=int, default=13, help="Number of classes")
    args = parser.parse_args()
    
    device = torch.device("cpu")
    model = CustomResNet50(args.num_classes)
    model.load_state_dict(torch.load(args.model_path, map_location=device))
    model.eval()
    
    dataset = MedmnistDataset(args.csv_path)
    loader = DataLoader(dataset, batch_size=53, shuffle=False)
    
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            y_true.extend(labels.numpy())
            y_pred.extend(preds.numpy())
    
    acc = accuracy_score(y_true, y_pred)
    print("Test Accuracy:", acc)
"""
with open("evaluate_model.py", "w") as f:
    f.write(eval_script)
print("Evaluation script saved as evaluate_model.py")


Evaluation script saved as evaluate_model.py


In [17]:
# Save test dataset as CSV
def save_split(X, y, path):
    # Reshape each image to a 1D vector and create a DataFrame
    df = pd.DataFrame(X.reshape((X.shape[0], -1)))
    df['label'] = y
    df.to_csv(path, index=False)
    print(f"Test CSV saved to {path}")

# Save the test set CSV to "split_data/test.csv"
os.makedirs('split_data', exist_ok=True)
save_split(X_test, y_test, "split_data/test.csv")


Test CSV saved to split_data/test.csv


In [24]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# Define the custom ResNet-50 model (same as used during training)
class CustomResNet50(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet50, self).__init__()
        self.resnet = models.resnet50(pretrained=False)
        # Modify the first convolution to accept 1-channel input
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Remove max pooling layer
        self.resnet.maxpool = nn.Identity()
        # Modify final layer for 13 classes
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
        
    def forward(self, x):
        return self.resnet(x)

# Define a dataset class that reads from CSV (each row is a flattened 28x28 image with a 'label' column)
class MedmnistDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)
        # Assume image pixels are stored as flattened values in all columns except 'label'
        self.X = df.drop('label', axis=1).values.reshape(-1, 1, 28, 28).astype(np.float32)
        self.y = df['label'].values.astype(np.int64)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

# Load the combined test set CSV
test_csv_path = "/kaggle/working/split_data/test.csv"  # update if needed
test_dataset = MedmnistDataset(test_csv_path)
test_loader = DataLoader(test_dataset, batch_size=53, shuffle=False)

# Load the saved model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomResNet50(num_classes=13)
model.load_state_dict(torch.load("best_custom_resnet50.pth", map_location=device))
model = model.to(device)
model.eval()

# Evaluate the model on the test set
y_true, y_pred, y_probs = [], [], []
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        probs = torch.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_probs.extend(probs.cpu().numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
try:
    auc = roc_auc_score(y_true, y_probs, multi_class='ovr', average='weighted')
except Exception as e:
    auc = None
    print("AUC computation error:", e)

# Print the evaluation metrics
print("Test Accuracy: {:.4f}".format(accuracy))
print("Test F1 Score: {:.4f}".format(f1))
print("Test Precision: {:.4f}".format(precision))
print("Test Recall: {:.4f}".format(recall))
print("Test AUC: {:.4f}".format(auc) if auc is not None else "Test AUC: Error computing AUC")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-24-2f8cc0f96ad0>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the function

Test Accuracy: 0.7450
Test F1 Score: 0.7060
Test Precision: 0.7438
Test Recall: 0.7450
Test AUC: 0.9821


In [21]:
from sklearn.metrics import classification_report

# Assuming y_true and y_pred are your true labels and predicted labels respectively
report = classification_report(y_true, y_pred, target_names=[f'Class {i}' for i in range(13)])

print(report)


              precision    recall  f1-score   support

     Class 0       0.62      0.95      0.75       250
     Class 1       0.89      0.83      0.86       250
     Class 2       0.98      0.34      0.50       250
     Class 3       0.79      0.96      0.87       250
     Class 4       0.98      0.69      0.81       234
     Class 5       0.84      0.99      0.91       390
     Class 6       0.45      0.99      0.62       174
     Class 7       0.00      0.00      0.00        46
     Class 8       0.00      0.00      0.00        92
     Class 9       0.58      0.16      0.25        68
    Class 10       0.00      0.00      0.00        20
    Class 11       0.73      0.52      0.61        42
    Class 12       0.84      0.91      0.87       114

    accuracy                           0.74      2180
   macro avg       0.59      0.56      0.54      2180
weighted avg       0.74      0.74      0.71      2180



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
